In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load dataset
df = pd.read_csv("auto.csv")

# Binary encoding for columns with 2 unique options
binary_cols = [col for col in df.columns if df[col].nunique() == 2]
for col in binary_cols:
    unique_values = df[col].unique()
    df[col] = df[col].apply(lambda x: 1 if x == unique_values[0] else 0)

# One-Hot Encoding for remaining categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Normalize numeric features
scaler = StandardScaler()
numeric_cols = ['km', 'age', 'hp_kW', 'Displacement_cc', 'Weight_kg', 'cons_comb']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Apply log transformation to the target
df['price'] = np.log1p(df['price'])

# Define features (X) and target (y)
X = df.drop('price', axis=1).values  # Predicting price
y = df['price'].values  # Target variable (price)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Perform Grid Search for Hyperparameter Tuning
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Display the best parameters and the corresponding score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {-grid_search.best_score_}")

# Use the best model from GridSearchCV
best_knn_model = grid_search.best_estimator_

# Train the KNN model
best_knn_model.fit(X_train, y_train)

# Predict on the test set
y_pred = best_knn_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_error(y_test, y_pred) / np.mean(y_test) * 100

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R²): {r2:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Best Parameters: {'n_neighbors': 5}
Best Score: 0.009056693574666128
Mean Squared Error (MSE): 0.0085
R-squared (R²): 0.9468
Mean Absolute Percentage Error (MAPE): 0.64%
